# Load Layer outputs

In [1]:
nn = 'NN'

import pickle
with open('results/' + nn + '_layer_outputs.dat','rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()

# Load recording and test data

In [2]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]


# downsample
factor = 1
if factor<1:
    Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtest.shape[0]):
        Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(X_pr.shape[0]):
        X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
else:
    Xtest_down = Xtest
    X_pr_down = X_pr
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

Using TensorFlow backend.


# Run XGBOOST

In [3]:
from copy import copy, deepcopy
from RE_PartialRecData import RE_PartialRecData
from RE_PartialRecData2 import RE_PartialRecData2
import os
import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier
import pickle

params = {}
# use softmax multi-class classification 'multi:softmax'
# use linear regression 'reg:linear'
params['objective'] = 'reg:linear'
# scale weight of positive examples
params['eta'] = 0.5               # Makes the model more robust by shrinking the weights on each step (0.01-0.2)
params['max_depth'] = 6           # Used to control over-fitting as higher depth will allow model to learn relations 
                                  # very specific to a particular sample. (3-10)
params['silent'] = 1
params['nthread'] = 4
# params['num_class'] = 10
num_round = 5

# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = [2**x for x in range(8)]
nSubnetSize = len(subnetSize)

# how many samples per recording?
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
# how many iterations
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])

for iLayer in range(0, len(layer_outs)-1, 2):
    print('Layer ', iLayer)#nSamples = np.divide(int(X_pr_down.shape[0]/recordings[nr]),subnetSize)*100
    for ss in range(nSubnetSize):
        print(subnetSize[ss])
        for it in range(nIterations):
            # copy data
            layer_outputs = deepcopy(layer_outs)
            # subsample
            X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], 
                                                          subnetSize[ss], nRecordings,nSamples[ss])
    #        print('# nan values: ',np.count_nonzero(np.isnan(X_subsample)))
            # prepare data for xgboost
            for iN in range(nOutNeurons):
                #print('#neuron, #iteratin, subnetsize: ', iN,it,subnetSize[ss])
                xg_train  = xgb.DMatrix(X_subsample, label=Y_subsample[:, iN])
                xg_test   = xgb.DMatrix(layer_outs_test[iLayer], label=layer_outs_test[oLayer][:,iN])
                watchlist = [(xg_train, 'train'), (xg_test, 'test')]
                # train XGboost
                bst = xgb.train(params, xg_train, num_round, watchlist, verbose_eval=False)
                # get predictions
                pred = bst.predict(xg_test)
                rmses[it, iN, ss] = np.sqrt(np.mean((bst.predict(xg_test)-layer_outs_test[oLayer][:,iN])**2))


            print ('predicting, RMSE=%f' %np.mean(rmses[it, :, ss]))


    fName = 'results/XGB_RMSES_Layer'+ str(iLayer) + 'nRec' + str(nRecordings) + nn + '.dat'
    # save the rmse's
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()

('Layer ', 0)
1
predicting, RMSE=0.270338
predicting, RMSE=0.272421
predicting, RMSE=0.275960
predicting, RMSE=0.263146
predicting, RMSE=0.261729
2
predicting, RMSE=0.246323
predicting, RMSE=0.257701
predicting, RMSE=0.254560
predicting, RMSE=0.247143
predicting, RMSE=0.244197
4
predicting, RMSE=0.224523
predicting, RMSE=0.235784
predicting, RMSE=0.228675
predicting, RMSE=0.242720
predicting, RMSE=0.239401
8
predicting, RMSE=0.232197
predicting, RMSE=0.230321
predicting, RMSE=0.233359
predicting, RMSE=0.234987
predicting, RMSE=0.226456
16
predicting, RMSE=0.212553
predicting, RMSE=0.213507
predicting, RMSE=0.210598
predicting, RMSE=0.216997
predicting, RMSE=0.219533
32
predicting, RMSE=0.195441
predicting, RMSE=0.192427
predicting, RMSE=0.189476
predicting, RMSE=0.193526
predicting, RMSE=0.175985
64
predicting, RMSE=0.132916
predicting, RMSE=0.136357
predicting, RMSE=0.141454
predicting, RMSE=0.138344
predicting, RMSE=0.140646
128
predicting, RMSE=0.106500
predicting, RMSE=0.106462
pre